In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
import kss
import re
import networkx as nx
import warnings
warnings.filterwarnings("ignore")

In [3]:
#Mecab 객체 생성
mecab = Mecab()

In [4]:
# df 불러오기
df = pd.read_csv("../test.csv", header=None)
df.columns = ['article']

In [5]:
# 중복 제거
df = df.drop_duplicates('article')
df.reset_index(inplace=True,drop=True)
print(df.shape)

(6224, 1)


In [8]:
#stopwords list화
temp = pd.read_csv("../stop_words.csv",header = None, usecols = [0], names = ['stopwords'])                                                
stopwords = {temp['stopwords'][i] : i for i in range(len(temp))}

In [10]:
stopwords

{'아': 0,
 '휴': 206,
 '아이구': 2,
 '아이쿠': 3,
 '아이고': 4,
 '어': 5,
 '나': 490,
 '우리': 569,
 '저희': 30,
 '따라': 393,
 '의해': 392,
 '을': 11,
 '를': 12,
 '에': 13,
 '의': 14,
 '가': 15,
 '으로': 16,
 '로': 17,
 '에게': 18,
 '뿐이다': 135,
 '의거하여': 390,
 '근거하여': 391,
 '입각하여': 22,
 '기준으로': 23,
 '예하면': 318,
 '예를 들면': 25,
 '예를 들자면': 26,
 '저': 27,
 '소인': 28,
 '소생': 29,
 '지말고': 31,
 '하지마': 32,
 '하지마라': 33,
 '다른': 332,
 '물론': 35,
 '또한': 129,
 '그리고': 225,
 '비길수 없다': 38,
 '해서는 안된다': 39,
 '뿐만 아니라': 40,
 '만이 아니다': 41,
 '만은 아니다': 42,
 '막론하고': 43,
 '관계없이': 44,
 '그치지 않다': 45,
 '그러나': 130,
 '그런데': 47,
 '하지만': 132,
 '든간에': 49,
 '논하지 않다': 50,
 '따지지 않다': 51,
 '설사': 52,
 '비록': 53,
 '더라도': 54,
 '아니면': 512,
 '만 못하다': 56,
 '하는 편이 낫다': 500,
 '불문하고': 58,
 '향하여': 59,
 '향해서': 60,
 '향하다': 61,
 '쪽으로': 62,
 '틈타': 63,
 '이용하여': 64,
 '타다': 65,
 '오르다': 66,
 '제외하고': 67,
 '이 외에': 68,
 '이 밖에': 69,
 '하여야': 70,
 '비로소': 71,
 '한다면 몰라도': 72,
 '외에도': 73,
 '이곳': 74,
 '여기': 582,
 '부터': 76,
 '기점으로': 77,
 '따라서': 78,
 '할 생각이다': 79,
 '하려고하다': 330,
 '이리하여':

## TextRank 모듈 사용

In [13]:
from textrank import KeysentenceSummarizer
def mecab_tokenizer(sent):
    words = mecab.pos(sent, join=True)
    words = [w for w in words if w.split('/')[0] not in stopwords]
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

summarizer = KeysentenceSummarizer(
    tokenize = mecab_tokenizer,
    min_sim = 0.3,
    verbose = False
)


In [14]:
for sentence in df['article']:
    try:
        a = kss.split_sentences(sentence)
        if len(a) < 10:
            continue
        keysents = summarizer.summarize(a, topk=3)
        print(keysents)
        print()
    except:
        print('error')
        print(sentence)
        break

[(22, 1.751348435841916, '이날 확진된 북구 거주 50대 여성 광주 72번 은 요양보호사로 근무 중이며 아직 감염경로가 불명확한 것으로 나타났다.'), (0, 1.679015592715821, '요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 7명…동작구 소재 문창초 전수검사 실시 1일 오후 광주 동구 동명동 CCC 아가페실버센터 에서 질병관리본부 관계자들이 확진자 음압병실 이송 절차를 밟기 위해 센터에 들어가고 있다.'), (1, 1.6285725709682173, '해당 센터는 코로나19 광주지역 46번 확진자가 요양보호사로 근무하는 곳이며 이날 센터 입소자 2명이 추가 확진 판정을 받았다.')]

[(1, 1.2112913500804048, '한국관광공사와 중국 최대 여행사 트립닷컴은 1일 한국 여행 상품을 라이브 커머스로 판매하는 행사를 진행했다.'), (0, 1.168015987007193, '한국관광공사와 중국 최대 여행사인 트립닷컴이 1일 함께 진행한 한국 여행 상품 판매 라이브 커머스에 200만명이 몰리며 흥행에 성공했다.'), (5, 1.1114318919724036, '슈퍼보스 라이브쇼는 트립닷컴의 공동 창업자 중 한 명인 량젠쟝 회장이 직접 출연해 해당 여행지를 소개하면서 호텔 숙박권과 관광상품의 할인 판매를 진행하는 판촉 이벤트다.')]

[(0, 1.6008671183212506, '금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 첫 100% 반환 결정 운용사가 고객 속였고 판매사는 검증 안 해 앵커 환매가 중단된 라임자산운용의 무역금융펀드에 대해 금융감독원이 사상 처음으로 투자자들에게 투자 원금 전액을 돌려주라는 결정을 내렸습니다.'), (3, 1.2717675670711461, '기자 금감원은 라임 무역금융펀드와 관련한 4건의 분쟁에 대해 판매사들이 2018년 11월 이후 펀드를 산 투자자에게 원금 전액을 돌려주라고 결정했습니다.'), (6, 1.18

[(0, 2.1991014519992538, '문재인 대통령.ⓒ청와대 문 대통령 지지율 47.5% 인국공 사태로 데드크로스’ 문재인 대통령의 지지율이 이른바 인국공 사태 인천국제공항공사 직고용 사태 와 6·17 부동산 대책으로 인해 14주 만에 데드크로스 부정평가가 긍정평가를 앞서는 상황 를 기록했다. 데일리안이 여론조사 전문기관 알앤써치에 의뢰해 실시한 7월 첫째 주 정례조사에서 문 대통령 국정 수행에 대한 긍정평가는 전주 49.1% 대비 1.6%p 소폭 내린 47.5%다. 반면 부정평가는 전주 44.7% 대비 3.3%p 오른 48.0%로 집계됐다. 추미애가 때릴수록 단단해지는 윤석열 윤석열 검찰총장이 차기 대선주자 선호도 조사에서 3위 로 올라섰다. 윤 총장이 리얼미터의 대선주자 지지도 조사 후보군에 포함된 것은 이번이 처음이다. 홍준표 전 의원과 오세훈 전 서울시장 등 범보수 진영에서 꾸준하게 이름을 올린 주자들의 지지율을 단숨에 뛰어넘었다. 리얼미터가 오마이뉴스 의뢰로 지난 22일부터 26일까지 조사한 결과에 따르면 차기 대선주자 선호도 조사에서 윤 총장 지지율은 10.1%를 기록했다. 데일리안 여론조사 인국공 사태 민주당 32.0%로 휘청 …통합당과 오차범위 더불어민주당과 미래통합당의 지지율 격차가 오차범위 이내로 좁혀졌다. 데일리안이 여론조사 전문기관 알앤써치에 의뢰해 실시한 7월 첫째주 정례조사에 따르면 더불어민주당 지지율은 32.0%로 전월대비 8.7%p 급락했다. 반면 미래통합당 지지율은 28.5%로 전월대비 9.5%p 급등했다. 문재인 집권세력의 늑대 본능 이솝우화에는 ‘이리’ 에피소드가 많다. 요즘 그중 하나가 자꾸 떠오른다. 어떤 양치기가 어린 이리를 입양해 자기 집의 양치기 개들과 함께 길렀다. 그런데 그 이리가 성장해 양치기개 역할을 하면서 원래 무리였던 이리들에게 양들을 이끌어 희생시켰다는 우화다. 요즘 우리나라 집권세력의 폭주가 가관이다. 폭주도 폭주지만 그 방향이 더 큰 문제다. 북한통치 스타일을 흠모하고 흉내 내려 안달인 것 같

[(3, 1.5313133469861135, '규제지역을 대폭 늘리고 각종 대출규제를 강화해 갭투자 차단에 주력한 6·17대책을 포함해 문재인정부는 출범 이후 모두 22차례 부동산 관련 대책을 내놨다.'), (14, 1.3528370253172888, '참여정부는 2002년 분양권 전매제한 부활 수도권 투기과열지구 지정 청약 자격제한 등을 골자로 한 5·23대책을 시작으로 ‘부동산 투기와의 전쟁’을 선포했다.'), (0, 1.340139243171282, '역대 정부 ‘집값 대책’ 살펴보니 각종 규제에도 집값 잡기 실패 유사 MB·朴정부는 규제 완화에 방점 찍어 김현미 국토교통부 장관. 연합뉴스 “지금까지 부동산 정책은 다 종합적으로 작동하고 있다.” 김현미 국토교통부 장관은 지난달 30일 국회 예산결산특위에서 정부의 부동산 대책이 효과적으로 작용하고 있다고 강조했다.')]

[(0, 3.1737597949399463, '‘집 스트레스’ 시달리는 중장년층 정부 “집값 잡겠다” 호언장담했지만 서울 아파트 값 1년 새 수억씩 올라 “더 늦기 전에”… 기를 쓰고 매매 집착 국민 자산 중 주택 비중 70% 달해 2018년 주택가격 상·하위 20% 10배差 10년새 오피스텔·고시원 거주비율 ‘쑥’ 공급 부족·단기대책에 ‘풍선효과’ 불러 “주거 취약층 고착화 우려… 지원 강화를” “결혼하고 열심히 청약을 넣었는데 당첨이 계속 안 됐다. 어느 순간 지치더라. 다 잊고 살다가 ‘이러다 정말 남들 다 가진 집 나만 못 갖고 은퇴하겠다’ 싶어서 덜컥 집을 샀다.” 서울의 12년차 직장인 A 42 씨. 그는 지난해 휩쓸리듯 유주택자 대열에 합류했다.'), (28, 1.6315725253150908, '연구팀은 “주거수준 하위가구에서 주택 이외의 거처 가구나 20대 가구주가 차지하는 비중이 증가하고 있는 점은 향후 주거 취약계층의 고착화를 가져올 가능성이 크기 때문에 이들에 대한 지원을 강화할 필요가 있다”고 강조했다.'), (22, 1.6122193966265606, '국토연구원

[(16, 1.5314036440017267, '경영계는 민주노총이 나중에라도 참여한다면 대화를 계속하겠단 입장 하지만 민주노총 상황이 불투명한 데다 한국노총은 이번 대화는 무산된 거라고 못박아 22년 만의 노사정 대타협은 합의안 서명만 남겨 놓고 불발 위기에 처했습니다.'), (4, 1.3878667001382605, '리포트 당초 예정된 노사정 대표자회의 합의문 서명식은 오늘 오전 10시 반. 서명식 한 시간 전 민주노총은 다시 한번 긴급회의를 소집했습니다.'), (1, 1.3349967454688496, '김명환 민주노총 위원장이 일부 조합원들의 반대로 서명식에 참석조차 못한 건데요.')]

[(0, 1.6651066672873922, '정정화 전 위원장 사퇴 후 5일만… 풍부한 경륜·역량 갖춘 적임자 월성 원전 맥스터 증설 놓고 갈등…산업부 차질없이 절차 진행 사용후핵연료 관리정책 재검토위원회의 신임 위원장으로 선출된 김소영 한국과학기술원 KAIST 교수. 사용후핵연료 재검토위원회 제공 © 뉴스1 세종 뉴스1 권혁준 기자 김소영 한국과학기술원 KAIST 과학기술정책대학원 교수가 사용후핵연료 관리정책 재검토위원회의 신임 위원장으로 선출됐다.'), (8, 1.4874904300116394, '정정화 사용후핵연료 관리정책 재검토위원회 전 위원장. 뉴스1 DB © News1 권현진 기자 정 전 위원장은 월성 원전 사용후 핵연료 임시저장시설 맥스터 의 증설을 둘러싼 시민 의견 수렴 과정에서 내부 구성원들과 마찰을 빚었다.'), (12, 1.406459609147669, '정 전 위원장은 사퇴 기자회견에서 이대로 반쪽 공론화를 강행한다면 뒤따르는 모든 책임은 산업부에게 돌아갈 것 이라면서 탈핵 시민단체를 포함한 이해당사자들이 포괄적으로 참여하고 대통령 직속 혹은 국무총리 산하기구 차원의 위원회를 꾸려야 한다고 주장하기도 했다.')]

[(16, 1.5444821215805824, '충전 업계 관계자는 “민간사업자가 이미 수만 기의 충전기를 운영하면서 전기차 인프라

[(16, 1.6779803249909762, '안병훈 GS리테일 생활서비스 MD부문장 상무는 GS리테일은 GS샵과 통합유통협의체를 신설해 양사의 역량을 극대화 할 수 있는 협업 시스템을 구축했다 라며 GS리테일은 GS샵과의 적극적인 협업을 통해 차별화 상품과 새로운 쇼핑 환경을 고객께 제공하는데 주력할 방침 이라고 말했다.'), (2, 1.5105587089054153, '1일 GS리테일과 GS샵은 양사의 상품 전문가 MD 로 구성된 통합유통협의체 를 신설하고 본격적인 활동을 시작했다고 밝혔다.'), (0, 1.464320807920517, 'GS리테일과 GSSHOP이 협업 조직 통합유통협의체 를 신설하고 공동 기획 상품을 내놨다.')]

[(7, 1.5543843038385918, '서울아파트 중위 전셋값은 문 대통령 취임 직전 같은 기간인 2014년 4월부터 2017년 5월 사이 2억9340만원에서 4억807만원으로 1억1467만원 올라 39.0% 상승했다.'), (0, 1.3954479177421417, '4억6129만원 조사 이래 최고치 3년 1개월간 13.0% 가파른 상승 강남 1위 이어 서초·송파 順 송파구 잠실동의 한 부동산 중개업소에 아파트 매매 및 전·월세 정보가 부착돼 있다.'), (3, 1.3569560243809033, 'KB국민은행은 1일 월간 주택가격 동향 시계열 자료를 내고서 올해 6월 서울 아파트 중위 전세가격이 4억6129만원을 기록했다고 밝혔다.')]

[(1, 1.3852301662552429, '현대건설 제공 디지털타임스 이상현 기자 현대건설은 건설업계 최초로 숙면환경 조성을 위한 침실 스마트 아트월 상품 H 슬리포노믹스 Sleeponomics 가칭 을 선보인다고 1일 밝혔다.'), (5, 1.2638575386514228, '현대건설은 H 슬리포노믹스 개발을 위해 브레인케어 전문회사인 지오엠씨와 이종업계 협업을 진행한다.'), (11, 1.1024234317648745, 'H 슬리포노믹스는 힐스테이트 갤러리 내에서 시범 

...

[(0, 2.193130169384732, '대규모 유상증자로 운용 수익률 상승 꾀한 듯 판매사 하이투자엔 부당이득금 반환 청구소송 제기 리보세라닙 NDA 등 하반기 모멘텀 기대 이데일리 김재은 기자 코스닥 시가총액 2위 바이오기업 에이치엘비 028300 가 환매중단 사태를 빚은 옵티머스자산운용 펀드에 400억원을 투자했다고 밝히며 논란이 일고 있다.'), (29, 1.7272665051144442, '에이치엘비 관계자는 “이번 유상증자를 추진하던 중 너무 안정적인 자금운용을 한다는 내부 문제제기가 있었다”며 “ 유증으로 적지 않은 자금이 조달되는 만큼 일부는 포트폴리오 분산 차원에서 사모펀드에 투자한 것”이라고 설명했다.'), (9, 1.7219049772764015, '◇ 300억은 블라인드펀드에 연 5% 수익 30일 금융투자업계에 따르면 에이치엘비는 지난 11일 하이투자증권을 통해 판매된 옵티머스자산운용의 블라인드펀드에 300억원을 위탁했다.')]

[(0, 2.18478916216081, '1997년 너무 일찍 나온 간편식 소스 캠핑족·1 2인가구 늘며 주목 죽 따로 소스 따로…편의성 떨어졌던 죽 파우치 형태로 맛·편의성 다 잡아 배달음식·홈술 늘며 탄산수도 쑥쑥 이데일리 김보경 기자 최근 사회 전반을 강타하고 있는 ‘깡’ 열풍. 가수 비가 2017년 12월에 발표했을 당시 혹평을 받았던 노래가 2020년 재조명을 받아 인기를 끌고 있다.'), (22, 1.5547401801930643, '시장 조사를 통해 일본 시장에서는 상품 죽이 파우치 형태가 대부분이라 용기 죽에만 국한된 국내 관점에서 탈피해 다양한 아이디어들을 도출할 수 있었고 죽 전문점에서 죽을 포장해 집에서 즐기는 수요가 증가하고 있는 상황도 캐치했다.'), (13, 1.5542317591656718, '캠핑족과 1 2인 가구가 늘어나면서 간편한 파우치 형태로 만든 다담이 입소문을 타기 시작한 것. CJ제일제당도 이들을 대상으로 집중적으로 마케팅을 한 결과 2012년 매출 200억원대 201

[(0, 1.8097246243383136, '이스타 인수 불발 가능성에 AK홀딩스 2거래일 간 7%대 상승 대형 항공주 2분기 실적호조 전망...“여객 수요 반등 따라줘야” 항공주의 2분기 실적 반등 가능성이 커졌지만 대내외적인 불확실성이 여전한 가운데 주가 상승에 대한 기대감이 줄어들고 있다.'), (1, 1.6164144071722437, 'ⓒ제주항공 신종 코로나바이러스 감염증 코로나19 사태 이후 화물 부문 매출 증가로 항공주의 2분기 실적 회복 가능성이 커졌다.'), (30, 1.568855528837084, '양지환 연구원은 “대한항공은 그동안 화물사업 호황기에 실적 개선과 주가 상승이 나타나는 경향이 높았다”며 “코로나19 치료제·백신 개발로 항공 업황이 돌아설 때 가장 빠르게 정상적인 영업활동이 가능할 수 있도록 준비하고 있다는 점도 긍정적 요인”이라고 짚었다.')]

[(26, 1.90563816381527, '금융투자업계 관계자는 정부의 증권세제 개편안이 스터디가 충분하지 않다는 느낌을 지울 수 없다 며 매월 소득금액을 원천징수하고 다시 돌려주는 방식이 자칫 단타를 유도하며 실무적으로 미수금이 발생하는 등 문제가 발생할 가능성이 높다고 본다 며 주식투자를 통해 얻는 복리 투자 소득을 크게 줄일 가능성도 없지 않다는 점에서 우려가 된다 고 지적했다.'), (0, 1.8630019073377095, '외국인 기관은 금융투자소득세 대상 제외 개인만 최고 25% 부과 이중과세 논란 매월 소득금액 통산 및 원천징수 주식투자 복리효과 떨어지고 단타로 변질 우려 오는 2023년부터 적용되는 금융투자소득세가 외국인과 기관투자가에 적용되지 않는 것으로 알려지면서 개인투자자들의 불만이 커지고 있다.'), (11, 1.5283235021889996, '지난 25일 기획재정부가 발표한 금융세제 선진화 추진 방향에 따르면 1년간 1월 1일 12월 31일 금융투자상품 소득·손실 금액을 과세표준으로 삼는데 매달 인별로 소득금액을 통산한 후 원천징수하는 방식을 2023년

[(8, 1.4677343100978053, '정부의 주식 양도소득세 과세 확대로 개인 투자자 개미 들의 성토가 연일 높아지는 가운데 “이대로 세금을 때려 맞을 순 없다”며 절세 방법을 모색하는 움직임도 빨라지고 있다.'), (12, 1.4526719406080244, '종목당 3억원 이상 보유한 투자자들은 ‘분산 투자’ 확대 전략을 찾는다.'), (18, 1.341388529025481, '수익이 비과세 2000만원 한도를 넘을 것 같다면 보유 종목 일부를 배우자에게 증여해 확정수익을 낮추는 식이다.')]

[(0, 1.763014611137125, '현대차 디지털 언박싱 통해 공개… 운전자 인식형 스마트 주행 적용 모델들이 30일 현대자동차의 중형 SUV 싼타페 부분변경 모델 ‘디지털 언박싱 행사’에서 포즈를 취하고 있다.'), (1, 1.2969939243749344, '새 플랫폼과 파워트레인 디자인을 적용해 신차급으로 재탄생한 신형 싼타페에는 진흙 눈 모래 등 다양한 주행 환경에 최적화된 ‘험로 주행 모드’도 추가됐다.'), (11, 1.123859869706501, '세계 최초로 적용된 ‘운전자 인식형 스마트 주행모드’도 눈에 띈다.')]

[(0, 1.918395902946394, '이마트 1인당 20장 롯데마트 5장… 편의점선 600원 수량 제한 없어 편의점 CU 관계자가 1일부터 판매할 비말차단용 마스크를 살펴보고 있다.'), (6, 1.6405950201987496, '먼저 이마트와 롯데마트는 비말차단용 마스크를 장당 500원에 판매하면서 인당 구매 제한을 뒀다.'), (13, 1.446600654555123, '이마트24는 에어퀸 2장·1950원 마스크를 함께 판매하고 세븐일레븐은 3일부터 ‘네퓨어 비말차단용 마스크’도 판매할 계획이라고 밝혔다.')]

[(0, 2.1284971695958497, '김포·파주 조정지역 세금개편 등 추가 규제…‘땜질식 처방’ 계속 투자 수익 환수 언급도…“자유시장경제주의 포기 발언” 김현미 국토교통부 장관이 지

[(0, 1.2132968313469892, '재난지원금 특수 소비는 늘어 5월 광공업 생산이 11년여 만에 가장 큰 폭으로 줄었다.'), (6, 1.1783568426687117, '제조업 재고율은 전달보다 8.6%포인트 상승한 128.8%까지 치솟아 1998년 8월 133.2% 이후 가장 높았다.'), (5, 1.1702619344018372, '제조업 평균가동률도 전달보다 4.6%포인트 하락한 63.6%로 2009년 1월 62.8% 이후 11년여 만에 최저 수준으로 떨어졌다.')]

[(11, 2.0458094730692817, '반도체로 버틴 2분기 이날 본지가 6월 16일 이후 나온 6개 증권사의 삼성전자 실적 전망치를 분석한 결과 2분기 예상 매출은 51조4900억원 영업이익은 6조3100억원으로 집계됐다.'), (0, 1.4331092256657603, '2분기 영업익 6조3100억 전망… 이 중 5조2600억이 반도체 몫 세계 메모리 반도체 3위인 미국의 마이크론 은 29일 현지 시각 깜짝 놀랄만한 올 3 5월 실적을 내놨다.'), (21, 1.3552755028006676, '이는 작년 2분기 반도체 영업이익 3조4000억원 보다 55% 증가한 것이다.')]

[(3, 1.5710663279113017, '제주 몽블필름 에서는 오는 8월31일까지 제주항공 탑승객에게 반려동물과의 스냅사진 촬영 20% 할인 혜택을 제공한다.'), (0, 1.5199570302270442, '국내선 6개 노선 항공권 프로모션 진행 서울 뉴시스 고은결 기자 제주항공은 반려동물과의 여행을 계획하는 펫팸족을 대상으로 제휴사 혜택을 제공한다고 30일 밝혔다.'), (1, 1.5148750434991372, '제주항공은 이날부터 오는 7월31일까지 판매하는 프로모션 항공권을 구매한 이들 중 반려동물과 함께 떠나는 고객에게 제휴사 혜택을 제공한다.')]

[(0, 2.3310194642723436, 'VIG 상조·영어교육업체 잇따른 투자 관심 E F PE는 환경폐기물 선택과 

[(0, 1.3593480349552687, '오늘 1일 부터 규제지역에서 주택담보대출을 받아 집을 사려면 6개월 안에 새로운 집에 전입해야 하며 주택 매매·임대사업자는 주택담보대출을 받을 수 없다.'), (1, 1.3384909639277751, '더팩트 DB 주택가격 관계없이 6개월 내 전입 의무 더팩트｜윤정원 기자 오늘 1일 부터 주택매매업과 임대업 사업자에 대해 주택담보대출이 금지된다.'), (13, 1.2995422607498535, '지금은 무주택자의 경우 투기지역 투기과열지구 내 시가 9억 원 초과 주택 구입을 위해 주택담보대출을 받게 되면 1년 내 전입해야 하고 조정대상 지역에선 2년 내 전입해야 한다.')]

[(0, 1.9993366452483854, '통계청 ‘5월 산업활동 동향’ 산업생산 5개월 연속 내리막 제조업 재고율도 외환위기 수준 나아질거라던 정부 전망 어긋나 긴급재난지원금 효과로 소비는 일시적으로 끌어올렸지만 경제 전반에 걸친 충격을 덜기에는 역부족이었다.'), (22, 1.856937382667343, '전 산업 생산 5개월 연속 감소. 그래픽 김현서 kim.hyeonseo12 joongang.co.kr 경기동행지수 순환변동치는 지난 5월 96.5였다.'), (8, 1.5933323002701405, '전 산업 생산 5개월 연속 감소. 그래픽 김현서 kim.hyeonseo12 joongang.co.kr 지난 5월 소매판매는 4.6% 전월 대비 늘었다.')]

[(0, 1.9430621051012373, '통계청 5월 산업동향 소비만 회복세 생산 1.2%·투자 5.9%·건설 4.3% 감소 경기동행지수 21년 4개월 만에 최악 제조업 부진탓..“추경·기업 지원 시급” 국회가 30일 오전 서울 여의도 국회에서 예산결산특별위원회 전체회의를 열었지만 미래통합당 의원들이 불참해 자리가 비어 있다.'), (5, 1.747155549034967, '◇제조업 재고율·경기지수 21년여 만에 최악 통계청은 30일 ‘2020년 5월 산업활동동향 이하

In [20]:
from textrank import KeywordSummarizer

summarizer = KeywordSummarizer(tokenize=mecab_tokenizer, min_count=2, min_cooccurrence=1)
summarizer.summarize(a, topk=5)

[('한국/NNP', 5.918603674338969),
 ('그룹/NNG', 5.666717716680068),
 ('테크놀로지/NNG', 4.973068711785303),
 ('사장/NNG', 4.040207008982723),
 ('경영/NNG', 3.7556005440322635)]

In [21]:
for i in range(len(df[0:10])):
    sentence = re.sub('[-=+,#/\?©:^$©@*\"※~&·%｜!』\\‘◇”|\(\)\[\]\<\>`\'’“…》.]', '', df['article'][i])
    a = kss.split_sentences(sentence)
    print([i[0].split('/')[0] for i in summarizer.summarize(a, topk=10)])
    

['코로나', '백신', '연구', '바이러스', '영국', '옥스퍼드', '중', '감염증', '뉴스', '시험']
['진행', '임상', '크리스탈', '지노', '믹스', '발티노스', '타트', '시험', '결과', '환자']
['동물', '반려', '공원', '저류지', '내', '자율', '쉼터', '운영', '신산', '산책']
['코로나', '확장', '경기', '제조업', '미국']
['포토레지스트', '생산', '삼성전자', '메모리', '일본', '기업', '반도체', '한국', '국내']
['대책', '검색어', '카페', '회원', '인터넷', '실시간', '만들', '내용', '부동산', '올리']
['약사', '대한', '약연', '수상자']
['확진', '번', '명', '광주', '감염', '요양', '이날', '대', '거주', '경로']
['확장', '경기', '코로나', '제조업', '미국']
['한국', '판매', '상품', '관광', '행사', '트립', '이날', '라이브', '진행', '숙박']


In [ ]:
# Word2Vec 모델링
# news_tokens = df['token'].tolist()  # data input as list

# params
v_dimension = 300
v_window = 8
model = Word2Vec.load('word2vec.model')

# model = Word2Vec(sentences = news_tokens, size = v_dimension, window = v_window, min_count = 5, workers = 4, sg = 0)